In [1]:
import pandas as pd
from sklearn.compose import ColumnTransformer 
from sklearn.impute import SimpleImputer
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator

In [2]:
import pandas as pd
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")

In [3]:
from pandas_profiling import ProfileReport
train_profile = ProfileReport(train_data, title="Train Data")
display(train_profile)

In [ ]:
import numpy as np
class CabinOneHotEncoder(BaseEstimator):

    def __init__(self):
        self.ct = []
        pass

    def fit(self, documents, y=None):
        return self

    def transform(self, X):
        def process(x):
            if isinstance(x, float) and np.isnan(x):
                return ("nan", 0, 0, 0)
            sp = x.split()
            pre = ""
            num = []
            for s in sp:
                if s[0] not in pre:
                    pre += s[0]
                if len(s)>1:
                    num.append(int(s[1:]))
            if len(num) == 0:
                num = [0]
            return (pre, len(sp), min(num), max(num))
        x = pd.DataFrame(X["Cabin"].map(lambda x: process(x)).tolist(), columns=["Cabin_Letter", "Cabin_split", "Cabin_min", "Cabin_max"])
        if len(self.ct) == 0:
            self.ct = x["Cabin_Letter"].unique()
            ct = self.ct
        else:
            ct = self.ct
        def ohe_Cabin(x):
            lt = [0] * len(ct)

            for c in x:
                for i, v in enumerate(ct):
                    if c in v:
                        lt[i] = 1
            return lt
        x2 = pd.DataFrame(x["Cabin_Letter"].map(lambda x: ohe_Cabin(x)).tolist(), columns=["Cabin_Letter_"+c for c in ct])
        return pd.concat([X, x, x2], axis=1).drop(columns=["Cabin", "Cabin_Letter"])
    
    


In [ ]:
print(np.where(train_data['Age'].isna())[0])
train_data.loc[[5,17,19]]
train_data[(train_data["Embarked"]=="Q") & (train_data["Pclass"]==3) & (train_data["SibSp"]==0)]

In [ ]:


# define the class OutletTypeEncoder
# This will be our custom transformer that will create 3 new binary columns
# custom transformer must have methods fit and transform
class PdOneHotEncoder(BaseEstimator):

    def __init__(self, categorical_cols):
        self.categorical_cols = categorical_cols
        pass

    def fit(self, documents, y=None):
        return self

    def transform(self, x_dataset):
        print(x_dataset)
        pd.get_dummies(x_dataset, columns = self.categorical_cols)
        
        return x_dataset

In [ ]:

cat_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="nan"),
    OneHotEncoder(handle_unknown="ignore", sparse=False)
)

pre_process = ColumnTransformer(remainder='passthrough',
                                transformers=[
                                    ('drop_columns', 'drop', ['Name', 'PassengerId', 'Ticket', "Embarked"]),
                                    ("simple_imputer_age", SimpleImputer(strategy="mean"), ["Age"]),
                                    ('cat_pipe', cat_pipe, ["Sex", "Pclass"]),
                                ])


In [ ]:
model_pipeline = Pipeline(steps=[
    ("cabin", CabinOneHotEncoder()),
    ('pre_processing',pre_process),
#     ('random_forest', RandomForestClassifier(max_depth=10,random_state=2))
])

In [ ]:
X = model_pipeline.fit_transform(train_data)
Y = train_data["Survived"]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
XX = CabinOneHotEncoder().transform(train_data)
print(XX.columns)
print(XX["Cabin_Letter_C"])
sns.scatterplot(x="Cabin_Letter_C", y="Survived", data=XX)
plt.show()

In [ ]:
# model_pipeline.fit(train_data, Y)
from sklearn.model_selection import GridSearchCV
param_grid = {
    "n_estimators": [10, 100, 200, 500],
    "max_depth": [10, 100, 200],
    "min_samples_split": [2,4, 7, 10]
}
clf = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, n_jobs=-1)
clf.fit(X, Y)
clf.best_score_, clf.best_estimator_

In [ ]:
import lightgbm as lgb
param_grid = {
    "n_estimators": [10, 100, 200, 500],
    "max_depth": [-5, 10, 100, 200],
    "learning_rate": [0.05, 0.09, 0.1, 0.2]
}
model = lgb.LGBMClassifier(random_state=42)
clf = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
clf.fit(X, Y)
clf.best_score_, clf.best_estimator_

In [ ]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")

In [ ]:
# SimpleImputer(strategy="mean").fit_transform(test_data[["Fare"]])
test_data.at[test_data["Fare"].isna(), "Fare"] = 9.33
tX = model_pipeline.transform(test_data)

In [ ]:
res = clf.predict(tX)
output = test_data[["PassengerId"]]
output["Survived"] = res
output.to_csv("predicted_v6.csv", index=False)
prev_out = pd.read_csv("predicted.csv")
sum(prev_out["Survived"] == output["Survived"])/output.shape[0]